In [449]:
import numpy as np
import itertools

In [450]:
# |0> = [1,0], |1> = [0,1], kronecker product used to form composite state basis
H = np.array([[1,1],[1,-1]])/np.sqrt(2)

In [451]:
def tensorProduct(lst):
    """
    Given a list of numpy arrays (or lists) finds the total tensor product 
    """
    prod = np.array(lst[0],dtype=np.complex_)
    for l in lst[1:]:
        prod = np.kron(prod,np.array(l,dtype=np.complex_))
    return prod

def getState(i,qNum): #better name
    """
    Gets the state corresponding to i in the binary convention (with n bits)
    """
    i = format(i,"b")
    
    if qNum-len(i) >= 0:
        s = tensorProduct([np.array([1,0]) for j in range(qNum-len(i))])
    else:
        i = i[:qNum]
        s = 1

    for d in i:
        if d == "0":
            s = np.kron(s,np.array([1,0]))
        elif d == "1":
            s = np.kron(s,np.array([0,1]))
    return s

def getNumber(s): 
    """
    Function that finds the associated number for a qubit tensor product state. 
    """
    HDim = s.shape[0]
    qNum = int(np.log2(HDim))
    for i in range(qNum):
        if np.allclose(getState(i,qNum) @ s, 1):
            return i
    
def extendOperator(A,i,qNum):
    """
    Takes a qubit operator and extends it to a n qubit space, the operator created acts on the ith tensor product.
    """
    return tensorProduct([np.eye(2) if j != i else A for j in range(qNum)])

In [452]:
def invQFT(f):
    """
    Performs inverse quantum fourier transform on qubit states.
    """

    HDim = f.shape[0]
    qNum = int(np.log2(HDim))
    fInv = f

    for i in range(qNum): 
        for k in range(2,qNum-i):
            cRk = np.eye(2,dtype=np.complex_)
            cRk[1,1] = np.exp(-2*np.pi*complex(0,1)/(2**k))
            cRkE = extendOperator(cRk,i,qNum)
            fInv = cRkE @ fInv
        
        #apply H
        HE = extendOperator(H,i,qNum)
        fInv = HE @ fInv
    
    return fInv


In [453]:
#test case for invQFT: 3 qubit system, |2> = |010> state should be the result
f = tensorProduct([[1,np.exp(np.pi*complex(0,0.5))],[1,-1],[1,1]])/np.sqrt(8) 
print(getNumber(invQFT(f)))


2


In [454]:

def phaseEstimation(t,U,e): 
    """
    Performs phase estimation.

    Inputs:
    t - accuracy of psi returned
    U - unitary matrix
    e - normalised ket
    """
    HDim = U.shape[0]

    def pow(n,prev=np.eye(HDim)):
        next = U @ prev
        if n == 1:
            return next
        return pow(n-1,next)
    
    f = tensorProduct([[1,0] for i in range(t)])


    for i in range(t):
        HE = extendOperator(H,i,t)
        f = HE @ f

        #apply 
        Ui = np.eye(2,dtype=np.complex_)
        Ui[1,1] = e @ pow(2**i) @ e
        UiE = extendOperator(Ui,i,t)
        f = UiE @ f
        
    return invQFT(f) #apply inverse QFT


In [455]:
#test for phase esimation, identity matrix, result should be 0
U1 = np.array([[1,0],[0,1]])
e1 = np.array([1,0])
getNumber(phaseEstimation(2,U1,e1))

0

In [456]:
def shor(N,qNum,phaseAttempts,orderAttempts,psiAcc):
    HDim = 2**qNum

    for i in range(orderAttempts):
        a = np.random.randint(2,high=N)
        K = np.gcd(a,N)
        if K != 1:
            return N/K,K
        
        #find U_a matrix
        U_a = np.zeros([HDim,HDim],dtype=np.complex_)
        for j,k in itertools.product(range(HDim), repeat=2):
            if j == a*k % N:
                U_a[j,k] = 1
        
        #random number generator models quantum probablities
        eigenVal, eigenVec = np.linalg.eig(U_a) #need to change to |1> thing
        rs = []
        for l in range(phaseAttempts):
            index = np.random.randint(0,high=HDim)
            phaseEst = getNumber(phaseEstimation(psiAcc,U_a,eigenVec[index]))
            if phaseEst != None:
                rs.append(index/phaseEst)
        
        if rs == []:
            continue

        R = np.lcm.reduce(rs)

        if R % 2 == 0:
            g = np.gcd(a**(R/2) + 1,N)
            if g != 1:
                return N/g, g
            
    return "failed to factorize"


In [457]:
shor(12,5,5,10,5)

(6.0, 2)